In [11]:
!ls ../data/

Train.zip         titanic-train.csv tmp


In [12]:
!file ../data/titanic-train.csv

../data/titanic-train.csv: ASCII text, with CRLF line terminators


In [13]:
!head ../data/titanic-train.csv

In [14]:
%load_ext autoreload

%autoreload 2
%matplotlib inline

In [15]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from IPython.display import display
from sklearn import metrics

/Users/sameenislam/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/sameenislam/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [30]:
df_raw = pd.read_csv('../data/titanic-train.csv', low_memory=False, index_col='PassengerId')

In [32]:
df_raw.tail().T

PassengerId,887,888,889,890,891
Survived,0,1,0,1,0
Pclass,2,1,3,1,3
Name,"Montvila, Rev. Juozas","Graham, Miss. Margaret Edith","Johnston, Miss. Catherine Helen ""Carrie""","Behr, Mr. Karl Howell","Dooley, Mr. Patrick"
Sex,male,female,female,male,male
Age,27,19,NaN,26,32
SibSp,0,0,1,0,0
Parch,0,0,2,0,0
Ticket,211536,112053,W./C. 6607,111369,370376
Fare,13,30,23.45,30,7.75
Cabin,NaN,B42,NaN,C148,NaN


In [33]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(df_raw.drop('Survived', axis=1), df_raw['Survived'])

ValueError: could not convert string to float: 'Braund, Mr. Owen Harris'

In [34]:
train_cats(df_raw)

In [35]:
df_raw['Name'].cat.categories

Index(['Abbing, Mr. Anthony', 'Abbott, Mr. Rossmore Edward',
       'Abbott, Mrs. Stanton (Rosa Hunt)', 'Abelson, Mr. Samuel',
       'Abelson, Mrs. Samuel (Hannah Wizosky)',
       'Adahl, Mr. Mauritz Nils Martin', 'Adams, Mr. John',
       'Ahlin, Mrs. Johan (Johanna Persdotter Larsson)',
       'Aks, Mrs. Sam (Leah Rosen)', 'Albimona, Mr. Nassef Cassem',
       ...
       'Yrois, Miss. Henriette ("Mrs Harbeck")', 'Zabour, Miss. Hileni',
       'Zabour, Miss. Thamine', 'Zimmerman, Mr. Leo',
       'de Messemaeker, Mrs. Guillaume Joseph (Emma)',
       'de Mulder, Mr. Theodore', 'de Pelsmaeker, Mr. Alfons',
       'del Carlo, Mr. Sebastiano', 'van Billiard, Mr. Austin Blyler',
       'van Melkebeke, Mr. Philemon'],
      dtype='object', length=891)

In [36]:
df_raw.isnull().sum().sort_index()/len(df_raw)

Age         0.198653
Cabin       0.771044
Embarked    0.002245
Fare        0.000000
Name        0.000000
Parch       0.000000
Pclass      0.000000
Sex         0.000000
SibSp       0.000000
Survived    0.000000
Ticket      0.000000
dtype: float64

In [37]:
df, y, nas = proc_df(df_raw, 'Survived')

In [39]:
df.head().T

PassengerId,1,2,3,4,5
Pclass,3,1,3,1,3
Name,109,191,354,273,16
Sex,2,1,1,1,2
Age,22,38,26,35,35
SibSp,1,1,0,1,0
Parch,0,0,0,0,0
Ticket,524,597,670,50,473
Fare,7.25,71.2833,7.925,53.1,8.05
Cabin,0,82,0,56,0
Embarked,3,1,3,3,3


In [40]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(df, y)
m.score(df, y)

1.0

_Hmm...a perfect score. Maybe the model has overfit, so we will use a validation set._

In [41]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

In [45]:
df['Name']

PassengerId
1      109
2      191
3      354
4      273
5       16
      ... 
887    549
888    304
889    414
890     82
891    221
Name: Name, Length: 891, dtype: int16

In [46]:
n_valid = 418
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((473, 11), (473,), (418, 11))

In [47]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    print(res) 

In [48]:
m = RandomForestClassifier(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m) 

CPU times: user 213 ms, sys: 32.8 ms, total: 246 ms
Wall time: 262 ms
[1.0, 0.8444976076555024]


Training set `r^2= 1.0` , Validation set `r^2=0.84`